In [1]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import *
import shutil
import difflib
from pulp import *
import openpyxl

today = date.today()
#today = today - datetime.timedelta(days=1)
today = today.strftime("%m.%d.20%y")
today

'01.28.2022'

In [2]:
## DKNG Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool'

shutil.move(downloads + "/"+ "DKSalaries.csv",
           contest_data_loc + "/"+ "DKSalaries.csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool")
dkng_file = shutil.copy("DKSalaries.csv", "DKSalaries_"+ contest + str(today)+".csv")
dkng_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries.csv")
dkng_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("DKSalaries.csv")
os.remove("DKSalaries_"+ contest + str(today)+".csv")

In [171]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/DraftKings'
daily_file = "DK NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("DK NBA Projections.csv")

In [172]:
## Joins the DKNG and ETR Files to include DKNG Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\DraftKings\ETR_Daily_' + str(today) + ".xlsx")
dkng = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\DKNG_Data\PlayerPool\DKSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              dkng[['Name','Roster Position', 'Name + ID']], 
              left_on='Player', 
              right_on='Name')

In [173]:
df.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position,Name + ID
0,LaMelo Ball,CHA,LAL,35.0,PG/SG,"8,400",50.6,11.0,60.2,46.6,MAIN,LaMelo Ball,PG/SG/G/UTIL,LaMelo Ball (21156772)
1,Russell Westbrook,LAL,CHA,34.5,PG,"8,600",50.8,10.4,60.9,29.4,MAIN,Russell Westbrook,PG/G/UTIL,Russell Westbrook (21156766)
2,Jusuf Nurkic,POR,HOU,32.0,C,"8,000",46.5,8.5,55.4,18.6,MAIN,Jusuf Nurkic,C/UTIL,Jusuf Nurkic (21156789)
3,Nickeil Alexander-Walker,NOP,DEN,30.0,PG/SG,"4,400",31.5,7.9,39.6,30.6,MAIN,Nickeil Alexander-Walker,PG/SG/G/UTIL,Nickeil Alexander-Walker (21157177)
4,Nikola Jokic,DEN,NOP,35.0,C,"12,600",63.7,7.3,72.6,5.6,MAIN,Nikola Jokic,C/UTIL,Nikola Jokic (21156689)


In [174]:
## Unnests Roster Positions into each Row (explode in Python) ##


## Update Salary to INT 
df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

df['Roster Position'] = df['Roster Position'].astype(str)
df['Roster Position'] = df['Roster Position'].str.split('/')

df.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position,Name + ID
0,LaMelo Ball,CHA,LAL,35.0,PG/SG,8400.0,50.6,11.0,60.2,46.6,MAIN,LaMelo Ball,"[PG, SG, G, UTIL]",LaMelo Ball (21156772)
1,Russell Westbrook,LAL,CHA,34.5,PG,8600.0,50.8,10.4,60.9,29.4,MAIN,Russell Westbrook,"[PG, G, UTIL]",Russell Westbrook (21156766)
2,Jusuf Nurkic,POR,HOU,32.0,C,8000.0,46.5,8.5,55.4,18.6,MAIN,Jusuf Nurkic,"[C, UTIL]",Jusuf Nurkic (21156789)
3,Nickeil Alexander-Walker,NOP,DEN,30.0,PG/SG,4400.0,31.5,7.9,39.6,30.6,MAIN,Nickeil Alexander-Walker,"[PG, SG, G, UTIL]",Nickeil Alexander-Walker (21157177)
4,Nikola Jokic,DEN,NOP,35.0,C,12600.0,63.7,7.3,72.6,5.6,MAIN,Nikola Jokic,"[C, UTIL]",Nikola Jokic (21156689)


In [175]:
df.to_excel("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx", index=False)
# os.system("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx")

# Create the Constraint Problem

Goal: Maximize DK Points

- Total Players = 8
- TotalSalary <= 50000
- TotalPosition_PG
- TotalPosition_SG
- TotalPosition_G
- TotalPosition_SF
- TotalPosition_PF
- TotalPosition_F
- TotalPosition_C
- TotalPosition_UTIL

In [184]:
prob = pulp.LpProblem('NBA_DFS_Daily', LpMaximize)

In [185]:
player_vars = [pulp.LpVariable(f'player_{row.Player}', cat='Binary') for row in df.itertuples()]

In [186]:
prob += pulp.lpSum(player_var for player_var in player_vars) == 8

In [187]:
# position constraints
def get_position_sum(player_vars, df, position):
    return pulp.lpSum([player_vars[i] * (position in df['Roster Position'].iloc[i]) for i in range(len(df))])
    

prob += get_position_sum(player_vars, df, 'PG') >= 1
prob += get_position_sum(player_vars, df, 'SG') >= 1
prob += get_position_sum(player_vars, df, 'G') >= 1
prob += get_position_sum(player_vars, df, 'SF') >= 1
prob += get_position_sum(player_vars, df, 'PF') >= 1
prob += get_position_sum(player_vars, df, 'F') >= 1
prob += get_position_sum(player_vars, df, 'C') >= 1
prob += get_position_sum(player_vars, df, 'UTIL') >= 1

# prob += get_position_sum(player_vars, df, 'PG') <= 3
# prob += get_position_sum(player_vars, df, 'PG') >= 1
# prob += get_position_sum(player_vars, df, 'SG') <= 3
# prob += get_position_sum(player_vars, df, 'SG') >= 1
# prob += get_position_sum(player_vars, df, 'SF') <= 3
# prob += get_position_sum(player_vars, df, 'SF') >= 1
# prob += get_position_sum(player_vars, df, 'PF') <= 3
# prob += get_position_sum(player_vars, df, 'PF') >= 1
# prob += get_position_sum(player_vars, df, 'C') <= 2
# prob += get_position_sum(player_vars, df, 'C') >= 1

In [188]:
# total salary constraint

prob += pulp.lpSum(df['DK Salary'].iloc[i] * player_vars[i] for i in range(len(df))) <= 50000

In [189]:
# finally, specify the goal
prob += pulp.lpSum([df['DK Points'].iloc[i] * player_vars[i] for i in range(len(df))])

In [190]:
# solve and print the status
prob.solve()
print(LpStatus[prob.status])

Optimal


In [191]:
# for each of the player variables, 
total_salary_used = 0
mean_AvgPointsPerGame = 0
for i in range(len(df)):
    if player_vars[i].value() == 1:
        row = df.iloc[i]
        print(row['Roster Position'], row.Player, row.Team, row['DK Salary'], row['DK Points'])
        total_salary_used += row['DK Salary']
        mean_AvgPointsPerGame += row['DK Points']
#mean_AvgPointsPerGame /= 9  # divide by total players in roster to get a mean
total_salary_used, mean_AvgPointsPerGame

['PG', 'SG', 'G', 'UTIL'] LaMelo Ball CHA 8400.0 50.6
['PG', 'G', 'UTIL'] Russell Westbrook LAL 8600.0 50.8
['C', 'UTIL'] Jusuf Nurkic POR 8000.0 46.5
['PG', 'SG', 'G', 'UTIL'] Nickeil Alexander-Walker NOP 4400.0 31.5
['C', 'UTIL'] Hassan Whiteside UTA 6900.0 40.0
['PG', 'SG', 'G', 'UTIL'] Jaylen Nowell MIN 4000.0 27.3
['SF', 'PF', 'F', 'UTIL'] Cameron Johnson PHX 4700.0 30.0
['SG', 'SF', 'F', 'G', 'UTIL'] Eric Gordon HOU 4800.0 29.3


(49800.0, 306.00000000000006)

In [154]:
df.dtypes

Player              object
Team                object
Opp                 object
Minutes            float64
DK Position         object
DK Salary          float64
DK Points          float64
DK Value           float64
DK Ceiling         float64
DK Own             float64
DK Slate            object
Name                object
Roster Position     object
Name + ID           object
dtype: object